In [ ]:
import cv2
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit
import time

# ==========================================
# CONFIGURACIÓN "MODO BESTIA" (PARA GPU L4)
# ==========================================
CONFIG = {
    # DIMENSIONES: Subimos a Full HD real o incluso 2K
    'width': 1920,
    'height': 1080,

    # SIMULACIÓN: ¡Duplicamos la población!
    # Con una L4, 10,000 partículas generan 100 millones de interacciones por frame.
    # La L4 se come esto en el desayuno.
    'num_particles': 12000,

    'num_types': 8,          # Añadimos más diversidad de colores
    'r_max': 120.0,           # Un poco más de radio para patrones más grandes
    'beta': 0.3,
    'friction': 0.80,        # Un poco más de fricción para controlar el caos de 12k partículas
    'dt': 0.05,

    # VISUALIZACIÓN
    'mode': 'tails',
    'tail_decay': 0.92,      # Estelas más largas (la memoria sobra)
    'point_size': 1,         # Puntos más finos para que se vea como "arena" líquida
    'duration_sec': 60       # Video más largo
}

# Añadimos más colores para los nuevos tipos (8 tipos)
COLOURS = jnp.array([
    [255, 0, 0],    # Azul
    [0, 255, 0],    # Verde
    [0, 0, 255],    # Rojo
    [255, 255, 0],  # Cyan
    [255, 0, 255],  # Magenta
    [0, 255, 255],  # Amarillo
    [255, 255, 255],# Blanco (Nuevo)
    [255, 128, 0]   # Naranja (Nuevo)
], dtype=jnp.uint8)

# ==========================================
# MOTOR FÍSICO (JAX/GPU)
# ==========================================

@jit
def update_physics(pos, vel, types, force_matrix, config_vals):
    # Desempaquetar constantes para JAX
    w, h = config_vals['width'], config_vals['height']
    r_max = config_vals['r_max']
    beta = config_vals['beta']
    friction = config_vals['friction']
    dt = config_vals['dt']

    # Separar coordenadas
    x = pos[:, 0]
    y = pos[:, 1]

    # --- 3. LÍMITES TOROIDALES (Cálculo de distancia) ---
    # Para que las fuerzas crucen los bordes, calculamos la distancia más corta
    # en un mundo envolvente.
    dx = x[:, None] - x[None, :]
    dy = y[:, None] - y[None, :]

    # "Truco" matemático para el toroide:
    dx = dx - jnp.round(dx / w) * w
    dy = dy - jnp.round(dy / h) * h

    dist_sq = dx**2 + dy**2
    dist = jnp.sqrt(dist_sq)

    # Evitar división por cero e identidad
    # Rellenamos la diagonal con infinito para que no se calculen fuerzas sobre sí mismo
    safe_dist = jnp.where(dist == 0, jnp.inf, dist)

    # Normalizar distancia (0 a 1 respecto al radio máximo)
    norm_dist = safe_dist / r_max

    # --- 2. LÓGICA DE FUERZAS POR PARTES (Piecewise Function) ---
    # Recuperamos la regla de atracción/repulsión de la matriz
    interaction_val = force_matrix[types[:, None], types[None, :]]

    # A. Zona de Repulsión (Cerca, d < beta)
    # Fuerza negativa fuerte que empuja hacia afuera
    repulsion = (norm_dist / beta) - 1.0

    # B. Zona de Interacción (Media, beta < d < 1)
    # Curva suave que sube y baja.
    # Fórmula: interaction * (1 - abs(2*d - 1 - beta)/(1-beta))
    interaction = interaction_val * (1.0 - jnp.abs(2.0 * norm_dist - 1.0 - beta) / (1.0 - beta))

    # Selección de fuerza según la distancia
    force = jnp.where(norm_dist < beta, repulsion, interaction)

    # C. Zona Lejana (d > 1) -> Fuerza Cero
    force = jnp.where(norm_dist >= 1.0, 0.0, force)

    # Calcular componentes de fuerza (Fuerza * Vector Dirección Unitario)
    fx = jnp.sum((dx / safe_dist) * force, axis=1)
    fy = jnp.sum((dy / safe_dist) * force, axis=1)

    # Factor de fuerza global (para que se muevan bien)
    fx *= r_max
    fy *= r_max

    # --- 1. GESTIÓN DE ENERGÍA Y FRICCIÓN ---
    # La velocidad se reduce en cada paso (multiplicar por friction < 1)
    vel_new = vel * friction + jnp.stack([fx, fy], axis=1) * dt

    # Actualizar posición
    pos_new = pos + vel_new * dt

    # --- 3. LÍMITES TOROIDALES (Actualización de posición) ---
    # Si salen por la derecha, entran por la izquierda (Módulo)
    pos_new = jnp.mod(pos_new, jnp.array([w, h]))

    return pos_new, vel_new

# ==========================================
# BUCLE PRINCIPAL
# ==========================================

def main():
    print(f"Generando simulación en modo: {CONFIG['mode'].upper()}")
    print("Inicializando GPU...")

    # Estados iniciales aleatorios
    key = jax.random.PRNGKey(42)
    pos = jax.random.uniform(key, (CONFIG['num_particles'], 2), minval=0, maxval=jnp.array([CONFIG['width'], CONFIG['height']]))
    vel = jax.random.uniform(key, (CONFIG['num_particles'], 2), minval=-1, maxval=1)
    types = jax.random.randint(key, (CONFIG['num_particles'],), 0, CONFIG['num_types'])

    # Matriz de fuerzas aleatoria (-1 a 1)
    # Esta matriz define el "ADN" del comportamiento
    matrix = jax.random.uniform(key, (CONFIG['num_types'], CONFIG['num_types']), minval=-1.0, maxval=1.0)

    # Empaquetar configuración para pasar a función JIT
    config_vals = {k: float(v) if isinstance(v, (int, float)) else v for k, v in CONFIG.items() if k not in ['mode', 'tail_decay', 'point_size', 'duration_sec']}

    # Configurar Video
    fps = 30
    total_frames = CONFIG['duration_sec'] * fps
    video_writer = cv2.VideoWriter('particle_life_advanced.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (CONFIG['width'], CONFIG['height']))

    # Canvas inicial (usamos float para mejor precisión en el desvanecimiento)
    canvas = np.zeros((CONFIG['height'], CONFIG['width'], 3), dtype=np.float32)

    # --- OPTIMIZACIÓN GPU (Variante de Alto Rendimiento) ---
    # Al usar JAX, estamos implícitamente usando la "Versión para GPU"
    # mencionada en el manual, lo que nos permite simular 5000+ partículas
    # sin necesidad de Spatial Hashing complejo en Python puro.

    for i in range(total_frames):
        if i % 30 == 0: print(f"Renderizando frame {i}/{total_frames}")

        # 1. Actualizar Física
        pos, vel = update_physics(pos, vel, types, matrix, config_vals)

        # Copiar datos a CPU para dibujar (NumPy)
        np_pos = np.array(pos)
        np_types = np.array(types)

        # --- VARIANTES VISUALES ---
        if CONFIG['mode'] == 'clear':
            # Limpiar pantalla totalmente
            canvas[:] = 0
        elif CONFIG['mode'] == 'tails':
            # Efecto FADING TAILS: Multiplicar por factor < 1 (oscurecer lo viejo)
            canvas *= CONFIG['tail_decay']
        elif CONFIG['mode'] == 'spaghetti':
            # ESPAGUETIS DE MAMÁ: No hacemos nada, el rastro se queda para siempre
            pass

        # Dibujar partículas
        # Convertimos posiciones a enteros
        coords = np_pos.astype(np.int32)

        # Vectorizamos el color para dibujar rápido
        particle_colors = np.array(COLOURS[np_types])

        # Dibujamos iterando (OpenCV es rápido en C++)
        # Nota: Para máxima velocidad en Python puro se podría pintar directo en matriz,
        # pero cv2.circle queda más bonito (antialiasing).
        canvas_uint8 = canvas.astype(np.uint8) # Copia temporal para dibujar

        for p in range(CONFIG['num_particles']):
            cv2.circle(canvas_uint8, (coords[p, 0], coords[p, 1]), CONFIG['point_size'], tuple(map(int, particle_colors[p])), -1)

        # Guardar en el buffer persistente (si estamos en modo tails/spaghetti)
        if CONFIG['mode'] != 'clear':
             canvas = canvas_uint8.astype(np.float32)

        # Escribir frame al video
        video_writer.write(canvas_uint8)

    video_writer.release()
    print("¡Hecho! Descarga 'particle_life_advanced2.mp4'")

if __name__ == '__main__':
    main()